In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score

In [2]:
merged = pd.read_csv('processed_data/ratings_with_tmdb.csv')

In [3]:
merged.head()

,userId,rating,tmdbId
0,1,1.0,197
1,1,4.5,10474
2,1,5.0,238
3,1,5.0,240
4,1,5.0,207


In [4]:
movie_sim_graph = pd.read_csv('processed_data/umap_movie_graph_truncated.csv')
movie_sim_graph.head()

,source,target,weight
0,2,1379,1.864265e-01
1,2,5971,2.804486e-07
2,2,7015,3.330669e-16
3,2,42112,1.000000e+00
4,3,7974,1.879906e-01


In [5]:
adj = movie_sim_graph.groupby('source').apply(
    lambda df: list(zip(df['target'], df['weight']))
).to_dict()

/var/folders/2s/dztmgkl93_xfxpgb_dc8lwhw0000gn/T/ipykernel_71419/1258435593.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  adj = movie_sim_graph.groupby('source').apply(


In [6]:
results = []
iterations = 50

for user, user_df in merged.groupby('userId'):
    mean_r = user_df['rating'].mean()
    user_df['label'] = (user_df['rating'] >= mean_r).astype(int)
    if user_df.shape[0] < 10:
        continue

    train = user_df.sample(frac=0.8, random_state=42)
    test  = user_df.drop(train.index)

    if len(test) < 5:
        continue

    beliefs = {}
    for _, r in train.iterrows():
        beliefs[r['tmdbId']] = float(r['label'])

    test_movies = test['tmdbId'].tolist()
    for m in test_movies:
        beliefs[m] = 0.5

    for _ in range(iterations):
        new_beliefs = beliefs.copy()
        for m in test_movies:
            neighbors = adj.get(m, [])
            if not neighbors: 
                continue
            
            num = 0; den = 0
            for nb, w in neighbors:
                if nb in beliefs:
                    num += w * beliefs[nb]
                    den += w
            
            if den > 0:
                new_beliefs[m] = num/den
        
        # check convergence
        if max(abs(new_beliefs[m]-beliefs[m]) for m in test_movies) < 1e-4:
            break

        beliefs = new_beliefs
    
    y_true = test['label'].values
    y_score = np.array([beliefs[m] for m in test_movies])
    y_pred = (y_score >= 0.5).astype(int)

    # Add class sizes
    results.append({
        'userId': user,
        'accuracy': accuracy_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_score) if len(np.unique(y_true))>1 else np.nan,
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'train_size': len(train),
        'test_size': len(test),
    })


results_df = pd.DataFrame(results)

/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rail/Library/Python/3.9/lib/python

In [7]:
results_df.head()

,userId,accuracy,f1,roc_auc,precision,recall,train_size,test_size
0,1,0.800000,0.857143,0.750000,1.000000,0.750,22,5
1,4,0.583333,0.736842,0.500000,0.636364,0.875,50,12
2,5,1.000000,1.000000,NaN,1.000000,1.000,21,5
3,7,0.545455,0.666667,0.700000,0.500000,1.000,42,11
4,8,0.608696,0.756757,0.607143,0.608696,1.000,90,23


In [8]:
results_df.to_csv('prc_user_based_results.csv', index=False)